In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno
from scipy.special import boxcox1p, inv_boxcox1p

from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
#from sklearn.model_selection import train_test_split,cross_validate,KFold,cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.metrics import r2_score,mean_squared_log_error,mean_squared_error,mean_absolute_error
import math
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor  
from sklearn.svm import SVR  
import xgboost as xgb
from ray.tune.sklearn import TuneSearchCV
from pickle import dump,load

def CategoryPreprocess(df):
    #hard code cat cols
    
    df.week_start_date = pd.to_datetime(df.week_start_date)
    df['month'] = df.week_start_date.dt.month
    
    cat_cols = ['month','weekofyear']
    dftransform=pd.get_dummies(data=df, columns=cat_cols)
    
    return dftransform


def NumericalPreprocess(df,process):
    
    #hard code numerical cols
    numerical_cols = ['ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
    
    
    df[numerical_cols] = df[numerical_cols].interpolate(method='linear', limit_direction='both')
    df['ndvi_avg'] = (df['ndvi_ne'] + df['ndvi_nw'] + df['ndvi_se'] + df['ndvi_sw']) / 4
    
    if process == 'train':
        scaler = MinMaxScaler()
        scaler.fit(df[numerical_cols])
        df[numerical_cols] = scaler.transform(df[numerical_cols])
        
        if df.city.iloc[0] == 'sj':
            dump(scaler, open('scaler_sj.pkl', 'wb'))
        elif df.city.iloc[0] == 'iq':
            dump(scaler, open('scaler_iq.pkl', 'wb'))
    
    elif process == 'infer':
        
        if df.city.iloc[0] == 'sj':
            scaler = load(open('scaler_sj.pkl', 'rb'))
        
        elif df.city.iloc[0] == 'iq':
            scaler = load(open('scaler_iq.pkl', 'rb'))
        
        df[numerical_cols] = scaler.transform(df[numerical_cols])
    
    
    return df  


def preprocess(df,process):
    
    df.drop(columns = ['reanalysis_sat_precip_amt_mm'], inplace = True )
    dfcat = CategoryPreprocess(df)
    dftransform = NumericalPreprocess(dfcat,process)
    
    return dftransform